Matt Malory gave me two file for m features that have the features for the manual and auto-trace cells. I am going to combine them together and use them as m features in the m-autoencoder

In [1]:
import numpy as np
import pandas as pd

In [2]:
path = "/home/fahimehb/Remote-AI-root/allen/programs/celltypes/workgroups/mousecelltypes/MachineLearning/Patchseq-Exc/dat/"
exc_file_manual = "/For_Fahimeh/ivscc_exc_RawFeatureWide.csv"
exc_file_auto = "/For_Fahimeh/ivscc_exc_autotrace_RawFeatureWide.csv"

inh_file_manual = "/For_Fahimeh/ivscc_inhib_RawFeatureWide.csv"
inh_file_auto = "/For_Fahimeh/ivscc_inhib_autotrace_RawFeatureWide.csv"

In [3]:
exc_manual = pd.read_csv(path + exc_file_manual)
exc_manual.specimen_id = exc_manual.specimen_id.astype(str)
exc_auto = pd.read_csv(path + exc_file_auto)
exc_auto.specimen_id = exc_auto.specimen_id.astype(str)

inh_manual = pd.read_csv(path + inh_file_manual)
inh_manual.specimen_id = inh_manual.specimen_id.astype(str)
inh_auto = pd.read_csv(path + inh_file_auto)
inh_auto.specimen_id = inh_auto.specimen_id.astype(str)

In [4]:
exc_auto.columns

Index(['specimen_id', 'apical_dendrite_bias_x', 'apical_dendrite_bias_y',
       'apical_dendrite_depth_pc_0', 'apical_dendrite_depth_pc_1',
       'apical_dendrite_depth_pc_2', 'apical_dendrite_depth_pc_3',
       'apical_dendrite_early_branch_path',
       'apical_dendrite_emd_with_basal_dendrite', 'apical_dendrite_extent_x',
       'apical_dendrite_extent_y', 'apical_dendrite_frac_above_basal_dendrite',
       'apical_dendrite_frac_below_basal_dendrite',
       'apical_dendrite_frac_intersect_basal_dendrite',
       'apical_dendrite_max_branch_order',
       'apical_dendrite_max_euclidean_distance',
       'apical_dendrite_max_path_distance', 'apical_dendrite_mean_contraction',
       'apical_dendrite_mean_diameter',
       'apical_dendrite_mean_moments_along_max_distance_projection',
       'apical_dendrite_num_branches',
       'apical_dendrite_num_outer_bifurcations',
       'apical_dendrite_soma_percentile_x',
       'apical_dendrite_soma_percentile_y',
       'apical_dendrite_s

In [5]:
print(exc_manual.shape, exc_auto.shape, inh_manual.shape, inh_auto.shape)
print("total number of cells: ", exc_manual.shape[0] + exc_auto.shape[0] + inh_manual.shape[0] + inh_auto.shape[0])

(384, 52) (783, 52) (519, 46) (823, 47)
total number of cells:  2509


In [6]:
exc_manual.columns == exc_auto.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True])

In [7]:
[c for c in inh_auto.columns if c not in inh_manual.columns]

['soma_surface_area']

In [8]:
# Since soma_surface_area is a feature that we wont use at the end, it is ok to drop it
# or add it and have it as nans in the other dataframe
inh_manual["soma_surface_area"] = np.nan

In [9]:
inh_manual.columns == inh_auto.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True])

In [10]:
def common_member(a, b):
    a_set = set(a)
    b_set = set(b)
 
    if (a_set & b_set):
        return (a_set & b_set)
    else:
        print("No common elements")

In [11]:
# Lets make sure that the specimen ids are unique

print(len(exc_auto.specimen_id))
print(len(np.unique(exc_auto.specimen_id)))

print(len(exc_manual.specimen_id))
print(len(np.unique(exc_manual.specimen_id)))

# Check if they are mutually exclusive
com_mem  = common_member(exc_manual.specimen_id.to_list(), exc_auto.specimen_id.to_list())
len(com_mem)

783
783
384
384


368

In [12]:
# Matt said, we will use the manual features when available! so I will drop the common cells from the auto file
exc_auto.drop(exc_auto[exc_auto['specimen_id'].isin(com_mem)].index, inplace = True)


In [13]:
# Lets make sure that the specimen ids are unique

print(len(inh_auto.specimen_id))
print(len(np.unique(inh_auto.specimen_id)))

print(len(inh_manual.specimen_id))
print(len(np.unique(inh_manual.specimen_id)))

# Check if they are mutually exclusive
com_mem  = common_member(inh_manual.specimen_id.to_list(), inh_auto.specimen_id.to_list())
len(com_mem)

823
823
519
519


491

In [14]:
inh_auto.drop(inh_auto[inh_auto['specimen_id'].isin(com_mem)].index, inplace = True)

In [15]:
exc_auto_manual = exc_auto.merge(exc_manual, how='outer')

In [16]:
inh_auto_manual = inh_auto.merge(inh_manual, how='outer')

In [17]:
# now lets check if inh and exc cells are mutually exclusive
common_member(inh_auto_manual.specimen_id.to_list(), exc_auto_manual.specimen_id.to_list())

No common elements


In [19]:
exc_auto_manual.head()

,specimen_id,apical_dendrite_bias_x,apical_dendrite_bias_y,apical_dendrite_depth_pc_0,apical_dendrite_depth_pc_1,apical_dendrite_depth_pc_2,apical_dendrite_depth_pc_3,apical_dendrite_early_branch_path,apical_dendrite_emd_with_basal_dendrite,apical_dendrite_extent_x,...,basal_dendrite_num_branches,basal_dendrite_soma_percentile_x,basal_dendrite_soma_percentile_y,basal_dendrite_stem_exit_down,basal_dendrite_stem_exit_side,basal_dendrite_stem_exit_up,basal_dendrite_total_length,basal_dendrite_total_surface_area,soma_aligned_dist_from_pia,soma_surface_area
0,586071558,15.640472,560.457128,114.138461,9.920419,38.424189,-30.054454,0.556559,63.294199,184.801397,...,16.0,0.419355,0.659323,0.000000,0.833333,0.166667,1596.996207,6516.402249,545.513147,409.064298
1,587062666,79.832357,512.398427,49.675234,0.431590,26.593156,-65.152302,1.258455,44.541769,171.111719,...,29.0,0.462483,0.687585,0.000000,1.000000,0.000000,2710.195284,28412.544422,515.275722,1180.761194
2,589716891,104.783264,577.102263,64.075500,-9.270648,41.876505,-25.428638,0.631016,63.050652,172.413453,...,15.0,0.000000,0.771772,0.000000,1.000000,0.000000,1234.382303,11339.125504,566.147737,421.849971
3,589754504,0.102719,70.813969,-11.217564,-14.783886,18.785130,5.232744,0.632779,10.314850,209.760225,...,18.0,0.459016,0.800546,0.000000,1.000000,0.000000,1630.137681,16149.448618,690.497947,783.824828
4,596995044,48.084711,591.383293,17.624717,-57.727757,69.308145,-21.146008,0.182827,33.485238,264.718481,...,20.0,0.485263,0.650526,0.142857,0.571429,0.285714,977.547590,5058.323976,679.123288,183.652931


In [20]:
inh_auto_manual.head()

,specimen_id,axon_bias_x,axon_bias_y,axon_depth_pc_0,axon_depth_pc_1,axon_depth_pc_2,axon_depth_pc_3,axon_depth_pc_4,axon_emd_with_basal_dendrite,axon_exit_distance,...,basal_dendrite_num_branches,basal_dendrite_soma_percentile_x,basal_dendrite_soma_percentile_y,basal_dendrite_stem_exit_down,basal_dendrite_stem_exit_side,basal_dendrite_stem_exit_up,basal_dendrite_total_length,basal_dendrite_total_surface_area,soma_aligned_dist_from_pia,soma_surface_area
0,530986537,23.183363,-120.081929,1711.138758,-977.430223,644.911883,353.597710,594.132624,4.085478,0.000000,...,98.0,0.432841,0.651864,0.000000,1.000000,0.000000,5622.446927,56230.745851,197.766115,31.838096
1,531241377,0.093554,-296.235655,2788.639563,-3425.812685,-2020.757167,3906.930034,2060.772370,33.672093,0.000000,...,34.0,0.321478,0.188668,0.250000,0.583333,0.166667,2464.674685,21382.053396,266.250840,23.956851
2,531982248,57.407327,-178.647279,1963.749593,-1143.717608,379.292836,480.329348,948.880379,19.596852,37.526923,...,69.0,0.489647,0.318280,0.000000,0.857143,0.142857,4954.184024,52830.961524,195.539016,36.197859
3,591626616,90.875908,121.796181,14187.657226,12532.437989,6351.349757,2256.045091,8440.919711,17.252390,0.000000,...,83.0,0.438596,0.137003,0.272727,0.545455,0.181818,4302.484383,35667.072860,275.274849,21.874919
4,591629193,102.586745,222.957515,-8756.528193,-2035.712946,8396.360104,5481.351398,10367.398914,9.853639,0.000000,...,43.0,0.495442,0.281194,0.428571,0.571429,0.000000,3339.172790,30131.305477,705.655050,25.918403


In [21]:
exc_auto_manual.to_csv(path + "/ivscc_exc_m_features_auto_manual.csv")
inh_auto_manual.to_csv(path + "/ivscc_inh_m_features_auto_manual.csv")